# Context Wrapup

## Process Destruction
* Free all resources
* Mark PCB as unused

* What else do we have to do?
    * Process depends on other processes
    * Options
        * Kill dependent processes
        * Return error
        * Allow processes to complete operation
        * Don't permit process termination

# System Calls
A way for a process to ask for a service from the kernel
* takes args from kernel
* returns a value - result/status of request
* calling process is **blocked**/stopped while syscall is serviced

## Mechanism
* Caller
    * passes args
* Kernel
    * gets and checks args (valid pointers? valid ranges? processes allowed to execute call?)
    * Performs request
    * Syscall param checking is super important because kernel has access to all memory and all privileged code
    
```c
syscall() {
    // Process params
    processParams(args);
    
    // Service the call. int is interrupt
    int x;
    
    // Return
    return result
}
```

### Process side - `syscall()` mechanism

* Each syscall has a stub function in the system lib (eg. `getpid()`, `read()`)
* Each stub calls a generic syscall routine
* Generic routine gets things ready for the kernel and executes the interrupt `int` instruction
```c
int read(int fd, void *buf, int len) {
    return syscall(SYS_READ, fd, buf, len);
}
```

some ways to implement:
```c
syscall(int, VAR_ARGS);

syscall0(int);
syscall1(int, arg1);
syscall2(int, arg1, arg2);
...
```
* Args
    * Put arguments on stack or into registers
    * Put REQ_ID on stack or into register
* Execute `int` instruction, argument is the interrupt number
* Get return value from `%eax`
* Return this value

### Argpassing
* Store each arg in general register
* Dispatcher must get args from context that was pushed onto stack
* This is very efficient, but there is only a limited number of registers. Dispatcher needs to know context layout


### Kernel side - context switcher and dispatcher
* Args must be manipulated by dispatcher, who knows count and type of args based on requestID
* Dispatcher responsible for checking args
* Checking+Validation must be done before using the memory


### tl;dr
* syscall
    * push args on stack
    * push reqID on stack
    * execute `int #`
* context switcher
    * push process context on stack
    * save stack ptr
    * switch stacks
    * load kernel context
    * return to dispatcher
* dispatcher
    * determine type of request
    * check+copy args
        * return error code as result code in PCB
        * service request and save return code
        * block process if needed
    * select next ready process
    * iret
* syscall
    * copy eax to variable
    * return variable